In [ ]:
from PIL import Image
import numpy as np
import time
import random
from mss import mss
import sys
import pynput.keyboard as kb
import pynput.mouse as m
import os
from datetime import datetime
from datetime import datetime

import logging

import cv2

from glob import glob

In [ ]:
def debug(*args, **kwargs):
    if verbose:
        print(*args, **kwargs)

def same_images(A, B, delta = 5):
    dist = np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))
    return dist < delta

def same_colors(a, b, delta = 10):
    dist = np.linalg.norm(a-b)
    return dist < delta


def _capture_portion_windows(x0, y0, w0, h0):
    x = x0/2 + WIN_OFFSET[0]
    y = y0/2 + WIN_OFFSET[1]
    w = w0/2 + 2
    h = h0/2 + 2

    with mss() as sct:
        monitor = {'left': int(x), 'top': int(y), 'width': int(w), 'height': int(h)}
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = image_to_numpy(img)
    
    img = img.repeat(2,axis=0).repeat(2,axis=1)
    img = img[x0%2:, x0%2:, :]
    img = img[:w0, :h0, :]
    return img

def _capture_portion_mac(x, y, w, h):
    x = x + MAC_OFFSET[0]
    y = y + MAC_OFFSET[1]

    with mss() as sct:
        monitor = {'left': int(x/2), 'top': int(y/2), 'width': int(w/2 + 1), 'height': int(h/2 + 1)}
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = image_to_numpy(img)

    img = img[x%2:, y%2:, :]
    img = img[:w, :h, :]
    return img

def capture_portion(x, y, w, h):
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    if PLATFORM == "windows":
        return _capture_portion_windows(x, y, w, h)
    else:
        return _capture_portion_mac(x, y, w, h)

def capture_screen():
    w, h = (1200, 900)
    return capture_portion(0, 0, w, h)

def capture_whole_screen():
    with mss() as sct:
        monitor = sct.monitors[1]
        sct_img = sct.grab(monitor)

    img = Image.frombytes('RGB', sct_img.size, sct_img.bgra, 'raw', 'BGRX')
    img = image_to_numpy(img)
    
    if PLATFORM == "windows":
        img = img.repeat(2,axis=0).repeat(2,axis=1)

    return img

def capture_pixel(x, y):
    return capture_portion(x, y, 1, 1)[0,0,:]

def show_image(arr, title = None):
    img = numpy_to_image(arr)
    if title is not None:
        img.show(title = title)
    else:
        img.show()

def save_image(arr, fname=None):
    if fname is None:
        fname = "saved/{}.png".format(time.time())
    img = numpy_to_image(arr)
    img.save(fname)


def numpy_to_image(arr):
    return Image.fromarray(arr.swapaxes(0, 1).astype(np.uint8), "RGB")

def image_to_numpy(img):
    return np.array(img).swapaxes(0, 1).astype(np.int32)[:,:,:3]

def image_to_cv2(img):
    img = np.uint8(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img

def cv2_to_image(img):
    img = img.astype(np.uint8)
    return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

def cv2_to_numpy(img):
    return image_to_numpy(cv2_to_image(img))

def numpy_to_cv2(arr):
    return image_to_cv2(numpy_to_image(arr))


def load_image(fname):
    img = Image.open(fname)
    return image_to_numpy(img)

def save_game_capture(fname=None):
    img = capture_screen()
    if fname is None:
        global game_start
        if game_start is None:
            game_start = datetime.now()
        debug(game_start)
        fname = game_start.strftime(f'saved/%Y-%m-%d %H.%M.%S.png')
    save_image(img, fname)


def calibrate_offset():
    screen = capture_whole_screen()

    screen_top = -1
    screen_bot = -1
    screen_right = -1
    screen_left = -1

    for x in range(0, screen.shape[0], 1208):
        stride = 0
        for y in range(screen.shape[1]):
            if same_colors(screen[x][y], [32, 41, 74], 5):
                stride += 1
            else:
                if stride == 908:
                    screen_top = y - stride + 4
                    screen_bot = y - 4
                elif stride == 4:
                    if screen_top == -1:
                        screen_top = y
                    elif screen_top == y - 904:
                        screen_bot = y - 4
                stride = 0

    for y in range(0, screen.shape[1], 908):
        stride = 0
        for x in range(screen.shape[0]):
            if same_colors(screen[x][y], [32, 41, 74], 5):
                stride += 1
            else:
                if stride == 1208:
                    screen_left = x - stride + 4
                    screen_right = x - 4
                elif stride == 4:
                    if screen_left == -1:
                        screen_left = x
                    elif screen_left == x - 1204:
                        screen_right = x - 4
                stride = 0
    
    debug("screen bounds: ({}, {}) to ({}, {})".format(screen_left, screen_top, screen_right, screen_bot))

    if screen_top != -1 and screen_bot != -1 and screen_right != -1 and screen_left != -1:
        if PLATFORM == "macOS":
            global MAC_OFFSET
            MAC_OFFSET = (screen_left, screen_top)
        else:
            global WIN_OFFSET
            WIN_OFFSET = (int(screen_left)/2, int(screen_top)/2)
    else:
        debug("trouble finding all edges of screen")


def is_game_over_screen():
    pixel = capture_pixel(680, 380)     # piggy bank
    return same_colors(pixel, [246, 196, 195], 10)

def is_arcade_screen():
    pixel = capture_pixel(170, 852)     # magenta arcade cabinet
    return same_colors(pixel, [143, 60, 148], 10)

def is_start_screen():
    pixel = capture_pixel(110, 30)     # dark yellow diagonal stripe
    return same_colors(pixel, [248, 205, 85], 10)

def is_great_play_screen():
    pixel = capture_pixel(290, 670)     # dark red gift box
    return same_colors(pixel, [172, 68, 59], 10)

def is_score_exceeds_screen():
    pixel = capture_pixel(330, 465)     # webkinz logo outline
    return same_colors(pixel, [36, 0, 141], 20)

def is_in_game_screen():
    pixel = capture_pixel(1055, 65)     # music note
    return same_colors(pixel, [179, 108, 247], 10)

def get_best_mask(target, masks):
    best_dist = -1
    best_idx = 0
    for i, mask in enumerate(masks):
        dist = np.mean(np.sqrt(np.sum(np.square(target - mask), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_idx = i
    return best_idx, best_dist

def get_best_mask_dict(target, masks):
    best_dist = -1
    best_key = 0
    for key, mask in masks.items():
        dist = np.mean(np.sqrt(np.sum(np.square(target - mask), axis = 2)))
        if best_dist == -1 or dist < best_dist:
            best_dist = dist
            best_key = key
    return best_key, best_dist

def calc_image_dist(A, B):
    return np.mean(np.sqrt(np.sum(np.square(A - B), axis = 2)))

def get_score(img = None):
    x,y = 195,29
    w,h = 24,35
    num_digits = 5
    
    if img is None:
        score_img = capture_portion(x, y, w*num_digits, h)
    else:
        score_img = img[x:x+w*num_digits,y:y+h,:]

    score_digits = []
    for i in range(num_digits):
        digit = get_best_mask(score_img[w*i:w*(i+1)], score_masks)[0]
        if digit == len(score_masks) - 1:
            digit = ''
        score_digits.append(digit)
    return int('0'+''.join(map(str, score_digits)))

def get_level(img = None):
    x,y = 736,29
    w,h = 24,35
    num_digits = 2

    if img is None:
        level_img = capture_portion(x, y, w*num_digits, h)
    else:
        level_img = img[x:x+w*num_digits,y:y+h,:]

    level_digits = []
    for i in range(num_digits):
        digit = get_best_mask(level_img[w*i:w*(i+1)], score_masks)[0]
        if digit == len(score_masks) - 1:
            digit = ''
        level_digits.append(digit)
    return int('0'+''.join(map(str, level_digits)))

def get_time(img = None):
    x,y = 466,31
    w,h = 22,32
    num_digits = 3

    if img is None:
        time_img = capture_portion(x, y, w*num_digits, h)
    else:
        time_img = img[x:x+w*num_digits,y:y+h,:]

    time_digits = []
    for i in range(num_digits):
        digit = get_best_mask(time_img[w*i:w*(i+1)], time_masks)[0]
        if digit == len(time_masks) - 1:
            digit = ''
        else:
            digit = digit % 10
        time_digits.append(digit)
    return int('0'+''.join(map(str, time_digits)))

def get_money(img = None):
    w,h = 24,35
    y = 29

    x = 901
    num_digits = 4

    if img is None:
        money_img = capture_portion(x, y, w*num_digits, h)
    else:
        money_img = img[x:x+w*num_digits,y:y+h,:]

    best_guess = None
    best_error = 1000

    money_digits = []
    error_1 = 0
    for i in range(num_digits):
        digit, error = get_best_mask(money_img[w*i:w*(i+1)], score_masks)
        if digit == len(score_masks) - 1:
            digit = ''
        money_digits.append(digit)
        error_1 += error
    money_digits = [x for x in money_digits if x != '']
    guess_1 = int('0'+''.join(map(str, money_digits)))

    num_digits -= 1

    money_digits = []
    error_2 = 0
    for i in range(num_digits):
        digit, error = get_best_mask(money_img[w*i+w//2:w*(i+1)+w//2], score_masks)
        if digit == len(score_masks) - 1:
            digit = ''
        money_digits.append(digit)
        error_2 += error
    money_digits = [x for x in money_digits if x != '']
    guess_2 = int('0'+''.join(map(str, money_digits)))

    if error_1 < error_2:
        return guess_1
    else:
        return guess_2


def press_key(key):
    keyboard.press(key)
    time.sleep(random.uniform(0.1, 0.15))
    keyboard.release(key)
    time.sleep(random.uniform(0.1, 0.15))
    
def click_mouse(x, y):
    if PLATFORM == "macOS":
        x = (x + MAC_OFFSET[0])/2
        y = (y + MAC_OFFSET[1])/2
    else:
        x = x + WIN_OFFSET[0]
        y = y + WIN_OFFSET[1]
    mouse.position = (x, y)
    time.sleep(random.uniform(0.2, 0.5))
    mouse.press(m.Button.left)
    time.sleep(random.uniform(0.1, 0.15))
    mouse.release(m.Button.left)
    time.sleep(random.uniform(0.1, 0.15))

def click_card(level, idx):
    cards = level_cards[level - 1]
    offset = level_offsets[level - 1]
    board = level_board[level - 1]
    pos = offset + (idx + 0.5) / cards * board
    click_mouse(pos[0], pos[1])

def capture_card(level, idx, rsize=None, asize=None, img=None, diff=False):
    cards = level_cards[level - 1]
    offset = level_offsets[level - 1]
    board = level_board[level - 1]
    if asize is None:
        if rsize is None:
            size = board / cards
        else:
            size = np.array([rsize, rsize]) * board / cards
    else:
        size = np.array([asize, asize])
    pos = offset + (idx + 0.5) / cards * board - size / 2
    if img is not None:
        pos[0], pos[1], size[0], size[1] = int(pos[0]), int(pos[1]), int(size[0]), int(size[1])
        pos = pos.astype(int)
        size = size.astype(int)
        card = img[pos[0]:pos[0]+size[0],pos[1]:pos[1]+size[1],:]
    else:
        card = capture_portion(pos[0], pos[1], size[0], size[1])
    if diff:
        pos[0], pos[1], size[0], size[1] = int(pos[0]), int(pos[1]), int(size[0]), int(size[1])
        pos = pos.astype(int)
        size = size.astype(int)
        return subtract(card, bg[pos[0]:pos[0]+size[0],pos[1]:pos[1]+size[1],:])
    return card

def subtract(A, B):
    img = np.stack((np.any(np.not_equal(A, B), axis=2),)*3, axis=-1)*A
    return img

def avg_color(img):
    return np.mean(img, axis=(0, 1))

def decode_board(lvl = 1):
    cards = level_cards[lvl - 1]
    board = np.empty(shape=[cards[0], cards[1]], dtype=object)
    diff = subtract(capture_screen(), bg)
    for i in range(cards[0]):
        for j in range(cards[1]):
            card = capture_card(lvl, np.array([i, j]), rsize = 1.0, asize = None, img = diff)
            card = crop_card(card)
            mini = minify_card(card)
            key, dist = get_best_mask_dict(mini, card_masks)
            cutoff = 120
            if dist > cutoff:
                print(f"{'='*100}\nWARNING: distance from ({cards[0]},{cards[1]}) to '{key}' = {dist} > {cutoff}\n{'='*100}")
            board[i,j] = key
    return board

def capture_card_wait(level, idx, sec=0.1, rsize=1.0):
    prev = None
    card = None
    start = time.time()
    while True:
        time.sleep(sec)
        card = capture_card(level, idx, rsize=rsize, asize=None, img=None, diff=True)
        if (card==prev).all():
            break
        prev = card
    print(time.time() - start)
    if card is None:
        return None
    key, dist = get_best_mask_dict(mini, card_masks)
    return key

def get_dominant_color(arr):
    img = numpy_to_image(arr)
    img.convert("RGB")
    img.resize((1, 1), resample=0)
    arr = image_to_numpy(img)
    return arr[0,0,:]

def find_known_pair(board):
    unmatched = {}
    pair = None
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            card = board[i,j]
            if card != "flipped" and card != "empty":
                if card in unmatched:
                    match = unmatched.pop(card)
                    return np.array([match, [i,j]])
                else:
                    unmatched[card] = np.array([i,j])
    return None

def find_unknown(board):
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            card = board[i,j]
            if card == "flipped":
                return np.array([i,j])
    return None

def count_remaining_matches(board):
    count = 0
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            card = board[i,j]
            if card != "empty":
                count += 1
    return count // 2

def find_partner(board, other):
    for i in range(0, board.shape[0]):
        for j in range(0, board.shape[1]):
            if (np.array([i,j]) == other).all():
                continue
            card = board[i,j]
            if card == board[other[0], other[1]]:
                return np.array([i,j])
    return None

def print_board(board):
    global EMOJIS
    global BACKGROUNDS
    global SYMBOLS

    output = ""
    for j in range(0, board.shape[1]):
        for i in range(0, board.shape[0]):
            card = board[i,j]
            if card not in SYMBOLS:
                animal, color = card.split("_")
                SYMBOLS[card] = f"\033[48;5;{BACKGROUNDS[color]['code']}m{EMOJIS[animal]}\033[0m"
            output += SYMBOLS[card]
        output += "\n"
    print(output)
    print()

# TODO: write this somewhere so don't have to calc each time
def update_level_info(level):
    diff = subtract(capture_screen(), bg)
    diff = diff[:,100:,:]   # ignore first 100 lines
    img = np.any(np.not_equal(diff, np.zeros([3])), axis=-1)
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    x,y,w,h = ymin, xmin+100, ymax-ymin+1, xmax-xmin+1
    level_offsets[level-1] = np.array([x,y])
    level_board[level-1] = np.array([w,h])

def asvoid(arr):
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))

def palette(img):
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]], count[order[::-1]]

def my_color_hash(img):
    colors, freqs = palette(img)
    hsh = set()
    i = 0
    num_colors = 3
    while len(hsh) < num_colors:
        if i > colors.shape[0] - 1:
            break
        else:
            color = colors[i,:]
            freq = freqs[i] / (img.shape[0]*img.shape[1])
            if freq < 0.02:
                i += 1
                continue
            # if (color==np.array([121,207,249])).all() or (color==np.array([0,25,214])).all():  # ignore light & dark blue
            if (color==np.array([0,25,214])).all():  # ignore dark blue
                i += 1
                continue
            hsh.add("%02x%02x%02x" % (colors[i,0], colors[i,1], colors[i,2]))
            i += 1
    return frozenset(hsh)

def init_board(lvl):
    cards = level_cards[lvl - 1]
    board = np.empty(shape=[cards[0], cards[1]], dtype=object)
    board.fill("flipped")
    if cards[0] % 2 == 1 and cards[1] % 2 == 1:
        board[cards[0]//2, cards[1]//2] = "empty"
    return board

def minify_card(img):
    return image_to_numpy(numpy_to_image(img).resize([63,63], Image.BILINEAR))

def test_number_parser():
    digit_tests = [
        {'path': 'saved/test_big.png', 'expected': {'score': 47859, 'time': 130, 'level': 24, 'money': 1399}},
        {'path': 'saved/1625982666.8275049.png', 'expected': {'score': 0, 'time': 4, 'level': 1, 'money': 0}},
        {'path': 'saved/1625982702.78526.png', 'expected': {'score': 22, 'time': 7, 'level': 2, 'money': 1}},
        {'path': 'saved/1625982713.5493178.png', 'expected': {'score': 67, 'time': 18, 'level': 3, 'money': 2}},
        {'path': 'saved/1625982729.036376.png', 'expected': {'score': 137, 'time': 6, 'level': 4, 'money': 4}},
        {'path': 'saved/1625982736.947087.png', 'expected': {'score': 150, 'time': 19, 'level': 4, 'money': 5}},
        {'path': 'saved/1625982756.895236.png', 'expected': {'score': 231, 'time': 5, 'level': 5, 'money': 7}},
        {'path': 'saved/1625982798.058444.png', 'expected': {'score': 368, 'time': 1, 'level': 6, 'money': 10}},
        {'path': 'saved/1625982837.043364.png', 'expected': {'score': 428, 'time': 8, 'level': 6, 'money': 13}},
        {'path': 'saved/1625982999.3064802.png', 'expected': {'score': 595, 'time': 11, 'level': 7, 'money': 17}}
    ]

    for test in digit_tests:
        img = load_image(test['path'])
        actual = {'score': get_score(img), 'time': get_time(img), 'level': get_level(img), 'money': get_money(img)}
        if actual != test['expected']:
            print(f"Failed test #{i} ({test['path']}):\n\tACTUAL:\t\t{actual}\n\tEXPECTED:\t{test['expected']}")
            

def crop_card(img):
    mask = np.logical_or(
        np.all(np.equal(img, BACKGROUNDS['pink']['rgb']), axis=-1),
        np.logical_or(
            np.all(np.equal(img, BACKGROUNDS['green']['rgb']), axis=-1),
            np.logical_or(
                np.all(np.equal(img, BACKGROUNDS['yellow']['rgb']), axis=-1),
                np.all(np.equal(img, BACKGROUNDS['blue']['rgb']), axis=-1)
            )
        )
    )
    if (mask == np.zeros([1])).all():
        return img
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    ymin, ymax = np.where(rows)[0][[0, -1]]
    xmin, xmax = np.where(cols)[0][[0, -1]]
    x,y,w,h = ymin, xmin, ymax-ymin+1, xmax-xmin+1
    w = h = max(w, h)
    return img[x:x+w, y:y+h, :]

def on_press(key):
    global break_program
    print("escape pressed")
    if key == kb.Key.esc:
        break_program = True
        return False

def log_level_info(msg = "MESSAGE", details = "details"):
    global prev_score, prev_time, prev_level, prev_money, logger
    img = capture_screen()
    curr_score, curr_time, curr_level, curr_money = get_score(img), get_time(img), get_level(img), get_money(img)
    score_inc, time_inc, level_inc, money_inc = curr_score - prev_score, curr_time - prev_time, curr_level - prev_level, curr_money - prev_money
    prev_score, prev_time, prev_level, prev_money = curr_score, curr_time, curr_level, curr_money
    logger.debug("\t".join([str(x) for x in [msg, details, level, curr_score, curr_time, curr_level, curr_money, score_inc, time_inc, level_inc, money_inc]]))
    return curr_score, curr_time, curr_level, curr_money

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# ch.setFormatter(formatter)
logger.addHandler(ch)
fh = logging.FileHandler("wacky.log")
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)

verbose = len(sys.argv) > 1 and sys.argv[1] == "-v"

# macOS, use native values rgb picker
# macOS, Safari: 16" MBP 2019, resolution scaled more space, resolution (4096x2560)
MAC_OFFSET = (761, 402)             # Safari (in top left)
# MAC_OFFSET = (1449, 797)          # Desktop app (centered)
MAC_DIMS = (1200, 900)
# windows: Microsoft edge, win+<-, my desktop, monitor (1920x1080)
WIN_OFFSET = (389, 230)
WIN_DIMS = (600, 450)

if os.name == "nt":
    PLATFORM = "windows"
elif os.name == "posix":
    PLATFORM = "macOS"

EMOJIS = None
SYMBOLS = None
BACKGROUNDS = None

keyboard = kb.Controller()
mouse = m.Controller()

game_start = None

calibrate_offset()

level_cards = [np.array([min(10,2+(i+1)//2), min(10,2+i//2)]) for i in range(100) ]

level_offsets = [
    np.array([256, 161]), 
    np.array([83,161]), 
    np.array([255,157]), 
    np.array([130,157])
] + [None for i in range(100)]

level_board = [
    np.array([692,692]), 
    np.array([1034,688]), 
    np.array([693,692]), 
    np.array([923,692])
] + [None for i in range(100)]



SYMBOLS = {"flipped": f"\033[48;5;21m❓\033[0m", "empty": f"\033[48;5;239m  \033[0m"}


EMOJIS = {
    "arte": "💎",
    "bunny": "🐰",
    "cat": "😸",
    "cow": "🐮",
    "dog": "🐶",
    "elephant": "🐘",
    "empty": "⬜️",
    "flipped": "🟦",
    "frog": "🐸",
    "gorilla": "🦍",
    "hippo": "🦛",
    "horse": "🐴",
    "lion": "🦁",
    "monkey": "🐵",
    "panda": "🐼",
    "pig": "🐷",
    "poodle": "🐩",
    "pug": "💀",
    "quack": "🦆",
    "unicorn": "🦄",
    "wacky": "⭐️"
}

BACKGROUNDS = {
    "pink": {"rgb": np.array([215, 163, 247]), "code": 171},
    "green": {"rgb": np.array([203, 250, 169]), "code": 77},
    "blue": {"rgb": np.array([121, 207, 249]), "code": 33},
    "yellow": {"rgb": np.array([255, 253, 84]), "code": 220},
}


# real = load_image("cards/all/levels_17+/empty.png")

# arr = load_image("cards/all/level_1/height_346_hsh_000000f8cd46.png")

# img = numpy_to_image(arr)
# img.show("ORIGINAL")

# small = img.resize([68,68], Image.NEAREST)
# # small.show(title="NEAREST")
# print("NEAREST", calc_image_dist(image_to_numpy(small), real))

# small = img.resize([68,68], Image.BOX)
# # small.show(title="BOX")
# print("BOX", calc_image_dist(image_to_numpy(small), real))

# small = img.resize([68,68], Image.BILINEAR)
# # small.show(title="BILINEAR")
# print("BILINEAR", calc_image_dist(image_to_numpy(small), real))

# small = img.resize([68,68], Image.HAMMING)
# # small.show(title="HAMMING")
# print("HAMMING", calc_image_dist(image_to_numpy(small), real))

# small = img.resize([68,68], Image.BICUBIC)
# # small.show(title="BICUBIC")
# print("BICUBIC", calc_image_dist(image_to_numpy(small), real))

# small = img.resize([68,68], Image.LANCZOS)
# # small.show(title="LANCZOS")
# print("LANCZOS", calc_image_dist(image_to_numpy(small), real))

# NEAREST with alert cutoff >100



time_masks = []
for i in range(10):
    img = Image.open('time/mask_' + str(i) + ".png")
    arr = image_to_numpy(img)
    time_masks.append(arr)
for i in range(6):
    img = Image.open('time/mask_' + str(i) + "r.png")
    arr = image_to_numpy(img)
    time_masks.append(arr)
img = Image.open('time/mask_blank.png')
arr = image_to_numpy(img)
time_masks.append(arr)

score_masks = []
for i in range(10):
    img = Image.open('score/mask_' + str(i) + ".png")
    arr = image_to_numpy(img)
    score_masks.append(arr)
img = Image.open('score/mask_blank.png')
arr = image_to_numpy(img)
score_masks.append(arr)

card_masks = {}
for filename in glob('cards/mini/*.png'):
    card = crop_card(load_image(filename))
    mini = minify_card(card)
    card_masks[filename[11:-4]] = mini

# card = load_image("cards/all/level_2/height_346_hsh_00000079cff9eb3323.png")
# card = crop_card(card)
# mini = minify_card(card)
# idx, dist = get_best_mask_dict(mini, card_masks)
# print(idx, dist)
# actual = card_masks["flipped"]
# print(calc_image_dist(actual, mini))

# show_image(mini)
# show_image(actual)
# show_image(card_masks[idx])
# show_image(crop_card(actual))

break_program = False

bg = load_image("background.png")

listener = kb.Listener(on_press=on_press)
listener.start()

MAX_SCORE = 30000
MIN_TIME = -1

click_mouse(0,0)

while not break_program:
    prev_score, prev_time, prev_level, prev_money = 0,0,0,0
    game_over = False

    if is_score_exceeds_screen():
        click_mouse(670, 520)
        time.sleep(0.5)
    if is_great_play_screen():
        click_mouse(470, 980)
        time.sleep(0.5)
    if is_game_over_screen():
        click_mouse(250, 700)
        time.sleep(0.3)
    if is_start_screen():
        click_mouse(460, 690)
        time.sleep(0.2)
    # TODO make exit if about to exceed, handle exceed and replay
    if not is_in_game_screen():
        print("Something is rotten in denmark")
        exit(1)

    level = get_level()
    log_level_info("NEW GAME", datetime.now().strftime("%m/%d/%Y %H:%M:%S"))

    while not break_program and not game_over:
        print(f"LEVEL: {level}")

        update_level_info(level)

        board = decode_board(level)

        print_board(board)

        while not break_program:
            curr_score = get_score()
            if curr_score + (level + 9) > MAX_SCORE:
                log_level_info("MAX SCORE", curr_score)
                click_mouse(1160, 45)                # exit mark
                time.sleep(3)
                game_over = True
                break
            curr_time = get_time()
            if curr_time < MIN_TIME:
                log_level_info("NO TIME", curr_time)
                click_mouse(1160, 45)                # exit mark
                time.sleep(3)
                game_over = True
                break

            print_board(board)
            known_pair = find_known_pair(board)
            # Flip known pairs
            if known_pair is not None:
                curr = known_pair[0]
                click_card(level, curr)
                prev = curr
                curr = known_pair[1]
                click_card(level, curr)
                time.sleep(0.3)
                log_level_info("KNOWN PAIR", board[curr[0], curr[1]])
                print("0: Known pair:", known_pair)
                board[curr[0], curr[1]] = "empty"
                board[prev[0], prev[1]] = "empty"
                print("\tMark empty ^:", known_pair)
            else:
                prev = None
                curr = None
                unknown = find_unknown(board)
                # Click unknown card
                if unknown is not None:
                    curr = unknown
                    click_card(level, curr)
                    capture_card_wait(level, curr, sec=0.1)
                    new_board = decode_board(level)
                    board[curr[0], curr[1]] = new_board[curr[0], curr[1]]
                    print("1: Unknown ", curr, board[curr[0], curr[1]])
                    partner = find_partner(board, curr)
                    # Click partner if known
                    if partner is not None:
                        prev = curr
                        curr = partner
                        click_card(level, curr)
                        time.sleep(0.3)
                        log_level_info("PARTNER FOUND", board[curr[0], curr[1]])
                        print("2: Known Partner ", curr)
                        board[curr[0], curr[1]] = "empty"
                        board[prev[0], prev[1]] = "empty"
                        print("\tMatch, mark empty ^:", curr, prev)
                    # Click another unknown
                    else:
                        unknown = find_unknown(board)
                        if unknown is None:
                            print("Warning, unmatched unknown")
                            break
                        prev = curr
                        curr = unknown
                        click_card(level, curr)
                        capture_card_wait(level, curr, sec=0.1)
                        new_board = decode_board(level)
                        board[curr[0], curr[1]] = new_board[curr[0], curr[1]]
                        print("2: Unknown ", curr, board[curr[0], curr[1]])
                        # Mark empty if incidental match
                        if board[curr[0], curr[1]] == "empty":
                            log_level_info("INCIDENTAL MATCH", board[prev[0],prev[1]])
                            board[prev[0], prev[1]] = "empty"
                        print("\tIncidental, mark empty ^:", curr, prev)
                else:
                    print("No more matches found")
                    log_level_info("LEVEL COMPLETE", level)
                    time.sleep(3)
                    level += 1
                    log_level_info("NEW LEVEL", level)
                    time.sleep(3)
                    break # moves to next level

        print_board(board)

    log_level_info("END OF GAME", datetime.now().strftime("%m/%d/%Y %H:%M:%S"))
log_level_info("BOT TERMINATED", datetime.now().strftime("%m/%d/%Y %H:%M:%S"))

In [9]:
img = capture_screen()


In [15]:
show_image(img)
board = decode_board()


In [14]:
level = 4